# A

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

#variables
dots = [(.5, 1.4), (2.3, 1.9), (2.9, 3.2)]
graph_x_list = np.linspace(0, 2)

#functions
def get_error_curve(slope):
    y_values = []

    for x in graph_x_list:
        y = 0

        for current_dot in dots:
            dot_x = current_dot[0]
            dot_y = current_dot[1]
            y += (dot_y - (x + (slope * dot_x))) ** 2
        
        y_values.append(y)
    
    return y_values


def get_gradient_line(intercept, slope):
    y_values = []

    for x in np.linspace(0, 4):
        y = intercept + (slope * x)
        y_values.append(y)
    
    return y_values


def get_tan(intercept, derivative, slope = .64):
    y_values = []

    y0 = 0 #error

    for current_dot in dots:
        dot_x = current_dot[0]
        dot_y = current_dot[1]
        y0 += (dot_y - (intercept + (slope * dot_x))) ** 2


    for x0 in graph_x_list:
        y = y0 + derivative * (x0 - intercept)
        y_values.append(y)
    
    return (y0, y_values)


def intercept_derivative_calculator(intercept, slope):
    derivative = 0
    for current_dot in dots:
        x = current_dot[0]
        y = current_dot[1]
        derivative += -2 * (y - (intercept + (slope * x)))
    
    return derivative


def slope_derivative_calculator(intercept, slope):


    pass


def gradient_descend(learning_rate = .1, step_size = 0, slope = .64, intercept = 0, minimum_step_size = .001, max_number_of_steps = 1000):
    animation_history = []
    tangent_list = []
    gradient_list = []
    curve_y_list = []

    step_count = 0
    while abs(step_size) > minimum_step_size or step_count == 0:
        current_history = []

        if step_count >= max_number_of_steps:
            break
        else:
            step_count += 1

        #getting the curve
        current_curve_y = get_error_curve(slope)
        curve_y_list.append(current_curve_y)

        #getting the derivative
        intercept_derivative = intercept_derivative_calculator(intercept, slope)

        #getting the tangent
        current_error, current_tangent = get_tan(intercept, intercept_derivative, slope)
        current_history.append(current_error) #current error

        #updating the step size
        step_size = intercept_derivative * learning_rate
        current_history.append(step_size) #step size
        
        #updating he intercept
        current_history.append(intercept) #old intercept
        intercept -= step_size
        current_history.append(intercept) #new intercept

        #updating the history
        animation_history.append(current_history) #[error, step_size, current_intercept, new_intercept] history
        tangent_list.append(current_tangent)
        gradient_list.append(get_gradient_line(intercept, slope))
    
    return (step_count, animation_history, tangent_list, gradient_list, curve_y_list)


def create_animation(step_count, animation_history, tangent_list, gradient_list, curve_y_list):
    fig, (ax_error, ax_gradient) = plt.subplots(1, 2, figsize=(14, 7))

    #error graph config
    ax_error.set_xlim(0, 2)
    ax_error.set_ylim(0, 4)
    ax_error.set_xticks([i * .25 for i in range(0, 9)])
    ax_error.set_yticks([i * .25 for i in range(0, 17)])
    ax_error.grid(True)
    ax_error.set_xlabel("Intercept", fontsize=12)
    ax_error.set_ylabel("Error", fontsize=12)
    
    #gradient graph config
    ax_gradient.set_xlim(0, 4)
    ax_gradient.set_ylim(0, 4)
    ax_gradient.set_xticks([i * .25 for i in range(0, 17)])
    ax_gradient.set_yticks([i * .25 for i in range(0, 17)])
    ax_gradient.grid(True)
    ax_gradient.set_xlabel("Weight", fontsize=12)
    ax_gradient.set_ylabel("Height", fontsize=12)

    #things on error graph
    error_curve, = ax_error.plot([], [], '-', color='blue')
    tangent_point, = ax_error.plot([], [], 'o', color='blue')
    tangent_line, = ax_error.plot([], [], '-', color='red')

    error_text = ax_error.text(0.010, 0.99, "", transform=ax_error.transAxes, fontsize=12, va='top')

    #things on gradient graph
    gradient_points, = ax_gradient.plot([], [], 'o', color='blue')
    gradient_line, = ax_gradient.plot([], [], '-', color='red')

    #updating the graph
    def update_animation(frame):
        history = animation_history[frame]
        error, step_size, current_intercept, new_intercept = history

        #error things
        error_curve.set_data(graph_x_list, curve_y_list[frame])
        tangent_point.set_data([current_intercept], [error])
        tangent_line.set_data(graph_x_list, tangent_list[frame])

        error_text.set_text(f"Step Size: {step_size:.2f}\nOld Intercept: {current_intercept:.2f}\nNew Intercept: {new_intercept:.2f}")

        #gradient things
        gradient_points.set_data([dot[0] for dot in dots], [dot[1] for dot in dots])
        gradient_line.set_data(np.linspace(0, 4), gradient_list[frame])

        return tangent_point, tangent_line, error_curve, gradient_points, gradient_line, error_text

    animation = FuncAnimation(fig, update_animation, frames=step_count, interval=100, blit=False)

    plt.close(fig)

    return animation


#plot
animation1 = create_animation(*gradient_descend(learning_rate=.1))
animation2 = create_animation(*gradient_descend(learning_rate=.2))

anim_list = [animation1, animation2]
html = "".join(anim.to_jshtml() for anim in anim_list)
HTML(html)